In [1]:
%load_ext autoreload
%autoreload 2

import requests
import re
from datetime import datetime
from src.models import CentrisUrl, BienCentrisDuplex, CustomDateFormat, Utilisation
from src.utils import extract_data_from_url
from src.centris_scraper import CentrisBienParser, CentrisScraper, ParsedUrl
import requests
from selectolax.parser import HTMLParser

In [2]:
plex_start_url="https://www.centris.ca/fr/plex~a-vendre~montreal?view=Thumbnail&uc=1"
url_bien = "https://www.centris.ca/fr/duplex~a-vendre~montreal-mercier-hochelaga-maisonneuve/26772323?view=Summary"

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url_bien, headers=headers)

if response.status_code == 200:
    tree = HTMLParser(response.text)

In [4]:
# inspect html
print(tree.html)

<!DOCTYPE html><html class="no-js" lang="fr" data-tenant="qc" data-user-authenticated="false" data-crawler="false"><head>
    <style>.async-hide { opacity: 0 !important }</style>
            <script>
                (function (a, s, y, n, c, h, i, d, e) {
                    s.className += ' ' + y;
                    h.start = 1 * new Date;

                    h.end = i = function () {
                        s.className = s.className.replace(RegExp(' ?' + y), '')
                    };

                    (a[n] = a[n] || []).hide = h;

                    setTimeout(function () {
                        i();
                        h.end = null
                    }, c);

                    h.timeout = c;
                })(window, document.documentElement, 'async-hide', 'dataLayer', 4000, { 'GTM-558V6M': true });
            </script>

    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
    <meta name="color-scheme"

In [5]:
parsed_url = ParsedUrl(url=url_bien, tree=tree)
data_parser = CentrisBienParser(parsed_url)

In [6]:
data_parser.carac_data

{'Utilisation de la propriété': 'Résidentielle',
 'Style de bâtiment': 'En rangée',
 'Année de construction': '1958',
 'Superficie du bâtiment (au sol)': '1 000 pc',
 'Superficie habitable': '2 000 pc',
 'Superficie du terrain': '2 025 pc',
 'Stationnement total': 'Allée (1), Garage (2)',
 'Nombre d’unités': 'Résidentiel (2)',
 'Unités résidentielles': '1 x 5 ½, 1 x 9 ½',
 'Unité principale': '9 pièces, 3 chambres, 2 salles de bain',
 'Revenus bruts potentiels': '40 800 $',
 'Caractéristiques additionnelles': 'Sous-sol 6 pieds +',
 'Date d’emménagement': 'Selon les baux'}

In [12]:
# data from URL
centris_id = data_parser.url_data.centris_id
ville = data_parser.url_data.ville
quartier = data_parser.url_data.quartier

# data from HTML
price = data_parser.extract_price()
description = data_parser.extract_description()
adresse = data_parser.extract_address()

annee_construction = data_parser.extract_annee_construction()
superficie_terrain = data_parser.extract_superficie_terrain()
superficie_commerce = data_parser.extract_superficie_commerce()

# stationnement = data_parser.extract_stationnement() #TODO: fix
style_batiment = data_parser.extract_style_batiment()
# nombre_unites = data_parser.extract_nombre_unites()
unites = data_parser.extract_unites()
revenus_brut = data_parser.extract_revenus()
eval_municipale = data_parser.extract_eval_municipale()
total_taxes = data_parser.extract_total_taxes()
# other_characteristics = data_parser.extract_additional_characteristics()


print(f"Price: {price}")
print(f"Description: {description}")
print(f"Address: {adresse}")
print(f"Construction Year: {annee_construction}")
print(f"Land Area: {superficie_terrain}")
# print(f"Stationnement : {stationnement}")
# print(f"Number of Units: {nombre_unites}")
print(f"Residential Units: {unites}")
print(f"Municipal Evaluation: {eval_municipale}")
print(f"Taxes: {total_taxes}")
print(f"Potential Gross Revenue: {revenus_brut}")
# print(f"Additional Characteristics: {misc}")


Price: 750000
Description: None
Address: 2810 - 2812, Avenue Parkville, Montréal (Mercier/Hochelaga-Maisonneuve), Quartier Mercier
Construction Year: None
Land Area: 2 025 pc
Residential Units: []
Municipal Evaluation: 535000
Taxes: 4000
Potential Gross Revenue: 40800


In [20]:
data_parser.extract_additional_characteristics()

: 

In [ ]:
total_row = tree.css_first('tr.financial-details-table-total')

# Extract the text from the <td> elements
if total_row:
    title_td = total_row.css_first('td.font-weight-bold')
    value_td = total_row.css_first('td.font-weight-bold.text-right')
    
    if title_td and value_td:
        title = title_td.text().strip()
        value = value_td.text().strip()
        print(f"{title}: {value}")
else:

    print("Total row not found.")

Total: 535 000 $


In [17]:
title_td.text().strip()

'Total'

In [ ]:
eval_total_div = soup.find('tr', class_='financial-details-table-total')
if eval_total_div and eval_total_div.find('td', class_='font-weight-bold text-right'):
    eval_municipale = int(re.sub(r'[^0-9]', '', eval_total_div.find('td', class_='font-weight-bold text-right').text.strip()))


In [ ]:
eval tree.css_first('tr.financial-details-table-total')
if eval_total_div:
    bold_cell = eval_total_div.css_first('td.font-weight-bold.text-right')
    if bold_cell and bold_cell.text():
        bien_centris['eval_municipale'] = int(re.sub(r'[^0-9]', '', bold_cell.text().strip()))

# Extracting taxes
yearly_taxes_div = tree.css_first('div.financial-details-table-yearly')
if yearly_taxes_div:
    taxes_total_div = yearly_taxes_div.css_first('tfoot tr.col.pl-0.financial-details-table-total')
    if taxes_total_div:
        bold_cell = taxes_total_div.css_first('td.font-weight-bold.text-right')
        if bold_cell and bold_cell.text():
            bien_centris['taxes'] = int(re.sub(r'[^0-9]', '', bold_cell.text().strip()))


Utilisation de la propriété: Résidentielle, Commerciale
Style de bâtiment: En rangée
Année de construction: 1912
Superficie commerciale disponible: 1 093 pc
Superficie du terrain: 1 677 pc
Nombre d’unités: Résidentiel (1), Commercial (1)
Unité résidentielle: 1 x 8 ½
Unité principale: 8 pièces, 3 chambres, 1 salle de bain
Revenus bruts potentiels: 45 900 $
Date d’emménagement: 30 jours après l’acceptation de la promesse d’achat ou de location


In [24]:
# Extracting municipal evaluation
eval_total_div = tree.css_first('tr.financial-details-table-total')
if eval_total_div:
    bold_cell = eval_total_div.css_first('td.font-weight-bold.text-right')
    if bold_cell and bold_cell.text():
        print(int(re.sub(r'[^0-9]', '', bold_cell.text().strip())))

# Extracting taxes
yearly_taxes_div = tree.css_first('div.financial-details-table-yearly')
if yearly_taxes_div:
    taxes_total_div = yearly_taxes_div.css_first('tfoot tr.col.pl-0.financial-details-table-total')
    if taxes_total_div:
        bold_cell = taxes_total_div.css_first('td.font-weight-bold.text-right')
        if bold_cell and bold_cell.text():
            print(int(re.sub(r'[^0-9]', '', bold_cell.text().strip())))

# Extracting additional characteristics
misc_div = tree.css('div:contains("Caractéristiques additionnelles")')
if misc_div:
    sibling = misc_div[0].next_element_by_css('.carac-value')
    if sibling and sibling.text():
        print(sibling.text().strip())

: 

In [23]:
tree

<HTMLParser chars=390952>

In [77]:
BienCentrisDuplex(centris_id = 1,
    url= CentrisUrl(url=url_bien),
    adresse = "",
    ville = "",
    quartier = "",
    annee_construction = 12,
    utilisation = Utilisation["Résidentiel"],
    garage = 0,
    prix = 12,
    eval_municipale = 12,
    taxes = 12,
    frais = 12,
    date_scrape = CustomDateFormat(date="2024-01-10"))

BienCentrisDuplex(centris_id=1, url=CentrisUrl(url='https://www.centris.ca/fr/duplex~a-vendre~montreal-le-plateau-mont-royal/21901946?view=Summary&uc=2'), adresse='', ville='', quartier='', annee_construction=12, utilisation=<Utilisation.Résidentiel: 'Résidentiel'>, superficie_habitable=None, superficie_batiment=None, superficie_terrain=None, garage=0, prix=12, eval_municipale=12, taxes=12, frais=12, revenus=None, description=None, date_scrape=CustomDateFormat(date='2024-01-10'))

In [ ]:
from playwright.sync_api import sync_playwright
playwright = sync_playwright().start()
browser = playwright.chromium.launch(headless=False)
page = browser.new_page()
page.goto("https://www.centris.ca/fr/plex~a-vendre~montreal?view=Thumbnail")
page.cl(path="example.png")
browser.close()
playwright.stop()